In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true" #tracing using langsmith
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [4]:
from langchain_groq import ChatGroq
llm_groq=ChatGroq(model_name="Gemma2-9b-It")

In [5]:
llm_groq.invoke("ISRO")

AIMessage(content="ISRO stands for **Indian Space Research Organisation**. \n\nHere's some key information about ISRO:\n\n* **What it is:** ISRO is the national space agency of India, responsible for the country's space program.\n* **Established:** 15 August 1969\n* **Headquarters:** Bengaluru, Karnataka, India\n* **Mission:** To harness space technology for national development, particularly in communication, education, meteorology, and resource management.\n* **Achievements:** ISRO has achieved numerous milestones, including:\n    * Launching its first satellite, Aryabhata, in 1975.\n    * Developing its own launch vehicles, such as the Polar Satellite Launch Vehicle (PSLV) and Geosynchronous Satellite Launch Vehicle (GSLV).\n    * Placing India's first human in space, Rakesh Sharma, in 1984.\n    * Successfully landing the Chandrayaan-3 spacecraft on the Moon in 2023, making India the fourth country to achieve a soft landing on the lunar surface.\n* **Future plans:** ISRO has ambiti

In [10]:
#give input to the model
message=[{"role":"system","content":"you are a helpful assistant"}, #model behaviour
         {"role":"user","content":"ISRO?"}] #question 

In [11]:
result = llm_groq.invoke(message)
print(result.content)

While I can provide information about ISRO and space exploration, I am not directly connected to ISRO or any government organization.

Think of me as a knowledgeable friend who has read a lot about space! I can answer your questions about ISRO's missions, history, achievements, and more.

What would you like to know about ISRO?



In [22]:
#For implementing agents i required a class called chatbot
class Chatbot:
    def __init__(self,system=""): #capture the message,system message
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message): #executing the llm
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        

In [23]:
bot = Chatbot(system="you are a helpful assistant")

In [24]:
bot("hi how are you?")

"As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today? 😊\n"

In [25]:
#history
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today? 😊\n"}]

In [21]:
bot.execute()

hi from execute fn


In [29]:
#React agent is purely based on the prompt so we have created the prompt specific to the react agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [31]:
#Regular expression
import re
action_re = re.compile('^Action: (\w+): (.*)')

Pattern Breakdown:

^: This matches the start of a string. It means the string must begin with what follows.

Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.

(\w+):

The parentheses () define a capture group. This allows you to extract part of the string that matches this section.

\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".

:: This is a literal colon that separates the word matched by (\w+) from the rest of the string.

(.*):

This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

What does it do?

This regex is looking for a string that:

Starts with the text "Action:".

Has a word right after it, separated by a colon.

Then, after another colon, it captures everything that follow

In [32]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [35]:
#Testing the pattern
match1 = action_re.match(text1)

In [36]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [37]:
match1.group(1), match1.group(2)

('Move', 'North')

In [38]:
match2 = action_re.match(text3)

In [39]:
match2.group(1)
#will give you an error

In [30]:
#wikipedia tool
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [26]:
#it a opensource database on top of that we implemented sql query 
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [71]:
def calculate(number):
    return eval(number)

In [72]:
calculate("20+27")

47

In [ ]:
#dictionary to match each and every tool
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [74]:
#logic for the react agent - responsible for either to search the query to LLM or to the Agents
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
    

In [75]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [76]:
bot = Chatbot(prompt)

In [77]:
bot("Hello!, how are you?")

'Hello! I am good. \n'

In [78]:
#query to the particular method
bot("What is your job?")

'Thought: I need to describe what I do. \n\nAction: simon_blog_search:  my purpose \nPAUSE \n'

In [79]:
next_prompt = "Tell me about Mahatma Gandhi?"

#passing the question to the bot

In [80]:
result = bot(next_prompt)

In [81]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [82]:
actions #empty string means it is referring the LLM and not the agents(wikipedia)

[<re.Match object; span=(0, 33), match='Action: wikipedia: Mahatma Gandhi'>]

In [83]:
next_prompt1 = "Tell me about the current situation of forest fire in Los Angeles from wikipedia?"

In [84]:
result1 = bot(next_prompt1)

In [85]:
actions1 = [action_re.match(a) for a in result1.split('\n') if action_re.match(a)]

In [86]:
actions1
#Here it is referring the wikipedia agent

[<re.Match object; span=(0, 47), match='Action: wikipedia: Forest fires in Los Angeles '>]

In [87]:
action, action_input = actions1[0].groups()

In [88]:
action

'wikipedia'

In [89]:
action_input

'Forest fires in Los Angeles '

In [90]:
#lets execute the custom bot
query("Hi")

Okay, I understand the process. I'm ready for my first question!  



"Okay, I understand the process. I'm ready for my first question!  \n"

In [91]:
query("How are you?")

Understood. I will follow the Thought, Action, PAUSE, Observation, Answer loop and use the available actions to respond to your questions.  

Let's begin! 



"Understood. I will follow the Thought, Action, PAUSE, Observation, Answer loop and use the available actions to respond to your questions.  \n\nLet's begin! \n"

In [92]:
query("twenty * twenty seven")

Thought: I need to calculate 20 * 27. 

Action: calculate: 20 * 27
PAUSE 

 -- running calculate 20 * 27
Observation: 540
Answer: 540 



'Answer: 540 \n'

In [93]:
#Lets ask related to simion blog
query("Has Simon have knowledge on AI?")

Thought: I need to search Simon's blog for information about AI. 
Action: simon_blog_search: AI
PAUSE 

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of 

"Answer: Based on Simon's blog post, it seems he is knowledgeable about AI. \n"

In [94]:
query("What is the capital of France?")

Thought: I should look up France on Wikipedia.
Action: wikipedia: France
PAUSE  

 -- running wikipedia France
Observation: <span class="searchmatch">France</span>, officially the <span class="searchmatch">French</span> Republic, is a country located primarily in Western Europe. Its overseas regions and territories include <span class="searchmatch">French</span> Guiana in
Thought: I need to keep looking for the capital. 
Action: wikipedia: France
PAUSE 

 -- running wikipedia France
Observation: <span class="searchmatch">France</span>, officially the <span class="searchmatch">French</span> Republic, is a country located primarily in Western Europe. Its overseas regions and territories include <span class="searchmatch">French</span> Guiana in
Thought: Hmmm, it seems like Wikipedia is having trouble giving me a complete answer.  Let me try a different approach. I'll search Simon's blog for information about France. 
Action: simon_blog_search: France
PAUSE 

 -- running simon_blog_search 

'Answer: The capital of France is Paris. \n'